In [1]:
# !pip install matplotlib==3.3.0

In [2]:
import tensorboard

In [3]:
tensorboard.__version__

'2.6.0'

In [4]:
import matplotlib.pyplot as plt
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
# Version
print(torch.__version__, torchvision.__version__)
data = pd.read_csv("./Data.csv")
idx = 0

1.9.1+cu111 0.10.1+cu111


In [6]:
# data['XMin'] = data['XMin']*255
# data['YMin'] = data['YMin']*255
# data['XMax'] = data['XMax']*255
# data['YMax'] = data['YMax']*255

In [7]:
# for i in tqdm(range(len(data))):
#     info = data.iloc[i]
# #     ymin,xmin,ymax,xmax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
#     height,width = cv2.imread(f'./Img/{info["Path"]}').shape[:2]
#     xmin,ymin,xmax,ymax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
#     print(xmin,ymin,xmax,ymax)
#     xmin = round(xmin * width)
#     xmax = round(xmax * width)
#     ymin = round(ymin * height)
#     ymax = round(ymax * height)
#     x = xmin
#     y = ymin
#     w = xmax - xmin
#     h = ymax - ymin
# #     x,y,w,h = x*255,y*255,w*255,h*255
#     x,y,w,h = round(x),round(y),round(w),round(h)
#     print(x,y,w,h,xmin,xmax,ymin,ymax,height,width)
#     img = cv2.imread(f'./Img/{info["Path"]}')
#     cv2.imwrite('./output.png',img)
#     roi=img[y:y+h,x:x+w]
#     cv2.imwrite(str('crop') + '.jpg', roi)
#     cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),10)
#     cv2.imwrite(str('box') + '.jpg', img)
#     plt.imshow(img)

In [8]:
# height

In [9]:
# width

In [10]:
# xmax - xmin

In [11]:
# img.shape

In [12]:
# info = data.iloc[50]
# # xmin,ymin,xmax,ymax = info['XMin']*im[0],info['YMin']*im[1],info['XMax']*im[0],info['YMax']*im[1]
# xmin,ymin,xmax,ymax = info['XMin']*255,info['YMin']*255,info['XMax']*255,info['YMax']*255
# # xmin,ymin,xmax,ymax = info['XMin'],info['YMin'],info['XMax'],info['YMax']
# x = xmin
# y = ymin
# w = xmax - xmin
# h = ymax - ymin
# # x,y,w,h = x*255,y*255,w*255,h*255
# # x,y,w,h = x*255,y*255,w*255,h*255
# x,y,w,h = round(x),round(y),round(w),round(h)

In [13]:
# Loading Data
def load_data(data=data, test=False):
    if test is True:
        if "data.npy" in os.listdir("./"):
            data = np.load("./data.npy", allow_pickle=True)
            np.random.shuffle(data)
            data = data[:325]
            print(len(data))
            return data
    if "data.npy" in os.listdir("./"):
        data = np.load("./data.npy", allow_pickle=True)
        np.random.shuffle(data)
        print(len(data))
        return data
    new_data = []
    for idx in tqdm(range(len(data))):
        record = {}
        info = data.iloc[idx]
        height, width = cv2.imread("./Img/" + info["Path"]).shape[:2]
        xmin, ymin, xmax, ymax = info["XMin"], info["YMin"], info["XMax"], info["YMax"]
        xmin = round(xmin * width)
        xmax = round(xmax * width)
        ymin = round(ymin * height)
        ymax = round(ymax * height)
        record["file_name"] = "./Img/" + info["Path"]
        record["height"] = height
        record["width"] = width
        record["cateogry_id"] = 0
        objs = [
            {
                "bbox": [xmin,ymin,xmax,ymax],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": 0,
            }
        ]
        record["image_id"] = idx
        record["annotations"] = objs
        new_data.append(record)
    np.random.shuffle(new_data)
    np.save("data.npy", new_data)
    return new_data

In [14]:
# Config
labels = ["Card"]

In [15]:
# Adding the data
DatasetCatalog.register("data", lambda: load_data())
MetadataCatalog.get("data").set(thing_classes=labels)
metadata = MetadataCatalog.get("data")
DatasetCatalog.register("test", lambda: load_data(test=True))
MetadataCatalog.get("test").set(thing_classes=labels)
metadata_test = MetadataCatalog.get("test")

In [16]:
# models = [
#     "fast_rcnn_R_50_FPN_1x.yaml",
#     "faster_rcnn_R_50_C4_1x.yaml",
#     "faster_rcnn_R_50_C4_3x.yaml",
#     "faster_rcnn_R_50_DC5_1x.yaml",
#     "faster_rcnn_R_50_DC5_3x.yaml",
#     "retinanet_R_50_FPN_1x.py",
#     "retinanet_R_50_FPN_1x.yaml",
#     "retinanet_R_50_FPN_3x.yaml",
#     "rpn_R_50_C4_1x.yaml",
#     "rpn_R_50_FPN_1x.yaml"
#     "faster_rcnn_R_50_FPN_1x.yaml",
#     "faster_rcnn_R_50_FPN_3x.yaml",
#     "faster_rcnn_R_101_DC5_3x.yaml",
#     "faster_rcnn_R_101_FPN_3x.yaml",
#     "faster_rcnn_X_101_32x8d_FPN_3x.yaml",
#     "",
# ]
model = f"COCO-Detection/retinanet_R_101_FPN_3x.yaml"
torch.cuda.empty_cache()
wandb.init(sync_tensorboard=True,name=f'{model}')
torch.cuda.empty_cache()
cfg = get_cfg()
torch.cuda.empty_cache()
cfg.merge_from_file(model_zoo.get_config_file(model))
torch.cuda.empty_cache()
cfg.DATASETS.TRAIN = ('data',)
torch.cuda.empty_cache()
cfg.DATASETS.TEST = ()
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)
torch.cuda.empty_cache()
cfg.SOLVER.MAX_ITER = 2500
torch.cuda.empty_cache()
# cfg.TEST.EVAL_PERIOD = 50
cfg.SOLVER.BASE_LR = 0.00025
torch.cuda.empty_cache()
cfg.SOLVER.STEPS = []
torch.cuda.empty_cache()
cfg.SOLVER.IMS_PER_BATCH = (1)
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(labels)
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
torch.cuda.empty_cache()
trainer = DefaultTrainer(cfg)
torch.cuda.empty_cache()
trainer.resume_or_load(resume=False)
torch.cuda.empty_cache()
trainer.train()
torch.cuda.empty_cache()
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.25
torch.cuda.empty_cache()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
torch.cuda.empty_cache()
predictor = DefaultPredictor(cfg)
torch.cuda.empty_cache()
evaluator = COCOEvaluator("test", output_dir="./output")
torch.cuda.empty_cache()
val_loader = build_detection_test_loader(cfg,"test")
torch.cuda.empty_cache()
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
torch.cuda.empty_cache()
wandb.log(metrics)
torch.cuda.empty_cache()
for img in os.listdir("./test_imgs/"):
    torch.cuda.empty_cache()
    v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
    torch.cuda.empty_cache()
    v = v.draw_instance_predictions(
        predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
    )
    torch.cuda.empty_cache()
    v = v.get_image()[:, :, ::-1]
    torch.cuda.empty_cache()
    plt.figure(figsize=(12, 6))
    torch.cuda.empty_cache()
    plt.imshow(v)
    torch.cuda.empty_cache()
    plt.savefig("./img.png")
    torch.cuda.empty_cache()
    plt.close()
    torch.cuda.empty_cache()
wandb.finish()

2021-10-15 09:22:53.176029: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.0
wandb: Run data is saved locally in wandb/run-20211015_092255-32hbmh6v
wandb: Syncing run COCO-Detection/retinanet_R_101_FPN_3x.yaml


Loading config /home/indika/anaconda3/envs/myenv/lib/python3.8/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.



[10/15 09:23:09 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
  anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


[10/15 09:23:10 d2.engine.train_loop]: Starting training from iteration 0


/home/indika/anaconda3/envs/myenv/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


[10/15 09:23:23 d2.utils.events]:  eta: 0:27:07  iter: 19  total_loss: 1.825  loss_cls: 1.472  loss_box_reg: 0.2783  time: 0.6642  data_time: 0.0350  lr: 4.9953e-06  max_mem: 3045M
[10/15 09:23:29 d2.utils.events]:  eta: 0:24:10  iter: 39  total_loss: 2.466  loss_cls: 1.896  loss_box_reg: 0.4625  time: 0.4623  data_time: 0.0013  lr: 9.9902e-06  max_mem: 3045M
[10/15 09:23:34 d2.utils.events]:  eta: 0:11:08  iter: 59  total_loss: 1.806  loss_cls: 1.384  loss_box_reg: 0.3704  time: 0.3880  data_time: 0.0013  lr: 1.4985e-05  max_mem: 3045M
[10/15 09:23:39 d2.utils.events]:  eta: 0:10:48  iter: 79  total_loss: 1.876  loss_cls: 1.563  loss_box_reg: 0.4795  time: 0.3500  data_time: 0.0012  lr: 1.998e-05  max_mem: 3045M
[10/15 09:23:43 d2.utils.events]:  eta: 0:10:25  iter: 99  total_loss: 1.525  loss_cls: 1.056  loss_box_reg: 0.3653  time: 0.3262  data_time: 0.0012  lr: 2.4975e-05  max_mem: 3045M
[10/15 09:23:48 d2.utils.events]:  eta: 0:10:07  iter: 119  total_loss: 1.183  loss_cls: 0.8316 

AssertionError: Results do not correspond to current coco set

In [ ]:
# cfg.MODEL.WEIGHTS = "./output/model_final.pth"
# predictor = DefaultPredictor(cfg)
# cfg.MODEL.WEIGHTS = "./output/model_final.pth"
# evaluator = COCOEvaluator("test", output_dir="./output")
# val_loader = build_detection_test_loader(cfg,"test")
# metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
# wandb.log(metrics)

In [ ]:
# # predictor = DefaultPredictor(cfg)
# cfg.MODEL.WEIGHTS = "./output/model_final.pth"
# evaluator = COCOEvaluator("test", output_dir="./output")
# val_loader = build_detection_test_loader(cfg,"test")
# metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
# wandb.log(metrics)

In [ ]:
for img in os.listdir("./test_imgs/"):
    v = Visualizer(cv2.imread(f"./test_imgs/{img}")[:, :, ::-1], metadata=metadata)
    v = v.draw_instance_predictions(
        predictor(cv2.imread(f"./test_imgs/{img}"))["instances"].to("cpu")
    )
    v = v.get_image()[:, :, ::-1]
    plt.figure(figsize=(12, 6))
    plt.imshow(v)
    plt.savefig("./img.png")
    plt.close()

In [ ]:
# evaluator = COCOEvaluator("data", output_dir="./output")
# val_loader = build_detection_test_loader(cfg,"data")
# metrics = inference_on_dataset(predictor.model, val_loader, evaluator)
# wandb.log(metrics)
# wandb.finish()

In [ ]:
# os.environ['TENSORBOARD_BINARY'] = '/path/to/envs/my_env/bin/tensorboard'

In [ ]:
# %load_ext tensorboard

In [ ]:
# import tensorflow as tf
# import datetime, os

In [ ]:
# %tensorboard --logdir output

In [ ]:
# %tensorboard --logdir outputs # TODO : remove this this is wrong.

In [ ]:
# !ls

In [ ]:
# from tensorboard import notebook

In [ ]:
# notebook.list() # View open TensorBoard instances

In [ ]:

# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("test", output_dir="./output")
# val_loader = build_detection_test_loader(cfg, "test")
# print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# predictor = DefaultPredictor(cfg)

In [ ]:
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("data", output_dir="./output")
# val_loader = build_detection_test_loader(cfg, "data")
# print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml'))
# cfg.DATASETS.TRAIN = ('data',)
# cfg.DATASETS.TEST = ()
# cfg.TEST.EVAL_PERIOD = 100
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025
# cfg.SOLVER.MAX_ITER = 2500
# cfg.SOLVER.STEPS = []
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
# predictor = DefaultPredictor(cfg)

In [ ]:
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader
# evaluator = COCOEvaluator("data", output_dir="./output")
# val_loader = build_detection_test_loader(cfg, "data")
# print(inference_on_dataset(predictor.model, val_loader, evaluator))